In [54]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sys
import nltk

# sklearn related
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
import pandas as pd
import numpy as np
import re 
import nltk 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold


from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
import matplotlib.patches as mpatches

In [56]:
train = pd.read_csv("./tweets/Corona_NLP_train.csv", encoding="latin1")
test = pd.read_csv("./tweets/Corona_NLP_test.csv", encoding="latin1")

In [57]:
def classes_def(x):
    if x ==  "Extremely Positive":
        return "2"
    elif x == "Extremely Negative":
        return "0"
    elif x == "Negative":
        return "0"
    elif x ==  "Positive":
        return "2"
    else:
        return "1"

In [58]:
train['text'] = train.OriginalTweet
train["text"] = train["text"].astype(str)

test['text'] = test.OriginalTweet
test["text"] = test["text"].astype(str)

In [59]:
train['label']=train['Sentiment'].apply(lambda x:classes_def(x))
test['label']=test['Sentiment'].apply(lambda x:classes_def(x))

In [60]:
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)
train['text_new']=train['text'].apply(lambda x:remove_urls(x))
test['text_new']=test['text'].apply(lambda x:remove_urls(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
train['text']=train['text_new'].apply(lambda x:remove_html(x))
test['text']=test['text_new'].apply(lambda x:remove_html(x))

# Lower casing
def lower(text):
    low_text= text.lower()
    return low_text
train['text_new']=train['text'].apply(lambda x:lower(x))
test['text_new']=test['text'].apply(lambda x:lower(x))


# Number removal
def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove
train['text']=train['text_new'].apply(lambda x:remove_num(x))
test['text']=test['text_new'].apply(lambda x:remove_num(x))

In [61]:
# import nltk
# nltk.download('stopwords')

In [62]:
#Remove stopwords & Punctuations
from nltk.corpus import stopwords

", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct
train['text_new']=train['text'].apply(lambda x:punct_remove(x))
test['text_new']=test['text'].apply(lambda x:punct_remove(x))


def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
train['text']=train['text_new'].apply(lambda x:remove_stopwords(x))
test['text']=test['text_new'].apply(lambda x:remove_stopwords(x))

In [63]:
#Remove mentions and hashtags
def remove_mention(x):
    text=re.sub(r'@\w+','',x)
    return text
train['text_new']=train['text'].apply(lambda x:remove_mention(x))
test['text_new']=test['text'].apply(lambda x:remove_mention(x))

def remove_hash(x):
    text=re.sub(r'#\w+','',x)
    return text
train['text']=train['text_new'].apply(lambda x:remove_hash(x))
test['text']=test['text_new'].apply(lambda x:remove_hash(x))

#Remove extra white space left while removing stuff
def remove_space(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove
train['text_new']=train['text'].apply(lambda x:remove_space(x))
test['text_new']=test['text'].apply(lambda x:remove_space(x))
test = test.drop(columns=['text_new'])
train = train.drop(columns=['text_new'])

In [64]:
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

In [75]:
X_train_raw = train["text"].tolist()
y_train = train["label"].tolist()
X_test_raw = test["text"].tolist()
y_test = test["label"].tolist()
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.10,
#                                                     random_state = 0)

# GloVe

In [66]:
# https://ithelp.ithome.com.tw/articles/10194633

In [67]:
glove = {}
f = open('glove.6B.100d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove[word] = coefs
f.close()

In [77]:
def glove_to_vec(data, dim=100):
    ret = np.zeros((len(data), dim))
    invalid = 0
    for n in range(len(data)):
        tweet = data[n]
        tokens = tweet.split()
        vecs = []
        for word in tokens:
            try:
                # throws KeyError if word not found
                vecs.append(glove[word])
            except KeyError:
                pass
        if len(vecs) > 0:
            vecs = np.array(vecs)
            ret[n] = vecs.mean(axis=0)
        else:
            invalid+=1
    return ret
    

# Random Forest

In [82]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, IsolationForest,VotingClassifier
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.neural_network import MLPClassifier

In [103]:
def create_grid_model(classifier, param_grid):
    cv = StratifiedShuffleSplit(n_splits=2, test_size=0.1)
    grid_model = GridSearchCV(classifier, param_grid=param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='accuracy')
    return grid_model

In [104]:
def evaluation(grid_model, predicted_test, test_label, predicted_train, train_label, file_name, 
           decision_function, clf_name="Classifier"):
    print("Results for ", clf_name, ": ")
    print()
    print("Best parameters are %s" % (grid_model.best_params_))
    acc_train = accuracy_score(train_label, predicted_train)
    acc_test = accuracy_score(test_label, predicted_test)
    print("Train Accuracy:  %0.3f" % (acc_train))
    print("Validation Accuracy:   %0.3f" % (grid_model.best_score_))
    print("Test Accuracy   %0.3f" % (acc_test ))


    print("Mean training time: %f" % (np.mean(grid_model.cv_results_['mean_fit_time'], axis=0)) )
    print("Mean test time: %f" % (np.mean(grid_model.cv_results_['mean_score_time'], axis=0)) )
    
    # confusion matrix
    print("Confusion matrix / precision recall scores")
    print ( confusion_matrix(test_label, predicted_test) )
    print ( classification_report(test_label, predicted_test) )
    
    
    f = open(file_name+'.txt','w')
    f.write("Best parameters are %s\n"% (grid_model.best_params_))
    f.write("Train Accuracy: %0.3f\n" % (acc_train))
    f.write("Validation Accuracy: %0.3f\n" % (grid_model.best_score_))
    f.write("Test Accuracy: %0.3f\n" % (acc_test ))
    f.write(str(confusion_matrix(test_label, predicted_test)) + "\n")
    f.write(str(classification_report(test_label, predicted_test)) + "\n\n")
    f.close()
    

In [105]:
# stock_with_absolute, predict 1 day trend 
rfc = RandomForestClassifier() # classifier
max_depth = [2, 4, 8, 10, 20]
n_estimators = [100, 500]#, 500, 1000]
max_features = [2 ,'sqrt']
# min_samples_split = [None, 10, 50]

# param to grid search
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features)

X_train = glove_to_vec(X_train_raw)
X_test = glove_to_vec(X_test_raw)

rfc_model = create_grid_model(rfc, param_grid)
rfc_model.fit(X_train, y_train)

predicted_test = rfc_model.predict(X_test)
predicted_train = rfc_model.predict(X_train)
decision_function = rfc_model.predict_proba(X_test)

evaluation(rfc_model, predicted_test, y_test, predicted_train, y_train, 
       "./results/rfc", decision_function, clf_name="RandomForest")

menyrbie phil_gahan chrisitv
abidsuleri shiffa_zy coronavirusinpakistan


coronavirusbetternetizensmockerymodemelaniastarpandemicadswatchshopstaffpushelderlymanoutsidestorelondonpubliclosingtempercovidpriyanka chopraurgefanstockupjoylovegrocerypandemic
usimposenewiranlinksanctionsentitiescovidconsumergoodgiantfaceboycottcall raisingsoaphandsanitiserpricescovidrbibuyblnworthgovtsecuritiesstabilisefinancialmarket
ynetalerts

coronacrisis coronavirusuk staysafe
coronacrisis


âcovidâ âgt covid_ covid springnews
covid_

germanymerkelquarantinesupermarketmeetingcovidinfecteddoctorussenatorrand paultestpositivejeffrey epsteinspentthousandsdollarsfundingallegedpimpghislainemaxwelldie

covidlockdowndontpanicthereismorefooddeputyagricminister
brazilhealthofficialconfirmfirstindigenouscoronaviruscaseamazonchiefbezossaydonatingfoodcharitycovidpandemictrumpcampaigndemandjeffsessioncampaignenddelusionalinvokingtiespresidentturkey



nhk_news
kedaceramicsghanaandsundainternationaldealersinfastmoving

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  6.3min finished


Results for  RandomForest : 

The best parameters are {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 500}
The Train Accuracy  0.997
The Validation Accuracy   0.653
The Test Accuracy   0.612
Mean training time: 53.676893
Mean test time: 0.254438
Confusion matrix / precision recall scores
[[ 998   64  571]
 [ 197  164  258]
 [ 333   52 1161]]
              precision    recall  f1-score   support

           0       0.65      0.61      0.63      1633
           1       0.59      0.26      0.36       619
           2       0.58      0.75      0.66      1546

    accuracy                           0.61      3798
   macro avg       0.61      0.54      0.55      3798
weighted avg       0.61      0.61      0.60      3798



In [102]:
evaluation(rfc_model, predicted_test, y_test, predicted_train, y_train, 
       "./results/rfc", decision_function, clf_name="RandomForest")

Results for  RandomForest : 

The best parameters are {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 500}
The Train Accuracy  0.725
The Validation Accuracy   0.616
The Test Accuracy   0.594
The mean training time of 104.324430
The mean test time of 0.417043
confusion matrix / precision recall scores
[[ 934   45  654]
 [ 194  122  303]
 [ 314   31 1201]]
              precision    recall  f1-score   support

           0       0.65      0.57      0.61      1633
           1       0.62      0.20      0.30       619
           2       0.56      0.78      0.65      1546

    accuracy                           0.59      3798
   macro avg       0.61      0.52      0.52      3798
weighted avg       0.61      0.59      0.57      3798

